Metadata
====

In [ ]:
import pathlib

parent_dir = pathlib.Path(
    "~/zebrafish_rdsf/Carran/Postgrad/Scale images from WT_spp1_sost/TIFs/segmentations"
).expanduser()
assert parent_dir.exists()

segmentation_paths = sorted([str(x) for x in parent_dir.glob("*.tif")])
f"{len(segmentation_paths)} segmentations"

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scale_morphology.scales import metadata

df = metadata.df(segmentation_paths)

# Replace NaN magnifications with -1
df.loc[np.isnan(df["magnification"]), "magnification"] = -1

def plot_mdata(df):
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    for axis, label in zip(axes, [c for c in df.columns if c != "path"]):
        axis.hist(df[label], bins=25)
        axis.set_title(label)
    
plot_mdata(df)

In [ ]:
plot_mdata(df[df["magnification"] != -1])